# LSTMレイヤの実装

In [1]:
import numpy as np
from common.functions import sigmoid

## [演習]
* 以下のLSTMレイヤのクラスを完成させましょう

In [2]:
class LSTM:
    def __init__(self, Wx, Wh, b):
        '''
        Parameters
        ----------
        Wx: 入力x用の重みパラーメタ（4つ分の重みをまとめたもの)
        Wh: 隠れ状態h用の重みパラメータ（4つ分の重みをまとめたもの）
        b: バイアス（4つ分のバイアスをまとめたもの）
        '''
        self.params = [Wx, Wh, b]
        self.grads = [np.zeros_like(Wx), np.zeros_like(Wh), np.zeros_like(b)]
        self.cache = None

    def forward(self, x, h_prev, c_prev):
        """
        順伝播計算
        """        
        Wx, Wh, b = self.params
        N, H = h_prev.shape

        A = np.dot(x, Wx) + np.dot(h_prev, Wh) + b

        f = A[:, :H]
        g = A[:, H:2*H]
        i = A[:, 2*H:3*H]
        o = A[:, 3*H:]

        f = sigmoid(f)
        g = np.tanh(g)
        i = sigmoid(i)
        o = sigmoid(o)
    
#         print(f.shape, c_prev.shape, g.shape, i.shape)
        c_next = f * c_prev + g * i
        tanh_c_next = np.tanh(c_next)
        h_next = o * tanh_c_next

        self.cache = (x, h_prev, c_prev, i, f, g, o, tanh_c_next)
        return h_next, c_next

    def backward(self, dh_next, dc_next):
        """
        逆伝播計算
        """        
        Wx, Wh, b = self.params
        x, h_prev, c_prev, i, f, g, o, tanh_c_next = self.cache

        A2 = (dh_next * o) * (1 - tanh_c_next ** 2)
        ds = dc_next + A2

        dc_prev = ds * f

        di = ds * g
        df = ds * c_prev
        do = dh_next * tanh_c_next
        dg = ds * i

        di *= i * (1 - i)
        df *= f * (1 - f)
        do *= o * (1 - o)
        dg *= (1 - g ** 2)

        dA = np.hstack((df, dg, di, do))

        dWh = np.dot(h_prev.T, dA)
        dWx = np.dot(x.T, dA)
        db = dA.sum(axis=0)

        self.grads[0][:] = dWx # 同じメモリ位置に代入
        self.grads[1][:] = dWh # 同じメモリ位置に代入
        self.grads[2][:] = db # 同じメモリ位置に代入

        dx = np.dot(dA, Wx.T)
        dh_prev = np.dot(dA, Wh.T)

        return dx, dh_prev, dc_prev

In [3]:
D = 10 # 入力データの次元
H = 5 # 中間層のノード数

Wx = (np.random.randn(D, 4 * H) / np.sqrt(D))
Wh = (np.random.randn(H, 4 * H) / np.sqrt(H))
b = np.zeros(4 * H)

# オブジェクトの生成
lstm = LSTM(Wx, Wh, b)

# 順伝播計算
N = 4 # バッチサイズ
x = np.random.randn(N, D)
h_prev = np.random.randn(N, H)
c_prev = np.zeros((N, H))
h_next = lstm.forward(x, h_prev, c_prev)
print("h_next=", h_next)
print()

# 逆伝播計算
dh_next = np.random.randn(N, H)
dc_next = np.random.randn(N, H)
dx, dh_prev, dc_prev = lstm.backward(dh_next, dc_next)
print("dx=", dx)
print()
print("dh_prev=", dh_prev)
print()


h_next= (array([[-0.01462164,  0.15747051, -0.23451682,  0.16172295,  0.00698562],
       [-0.06765944,  0.05940122, -0.57548391, -0.0059455 ,  0.26608388],
       [ 0.2789457 ,  0.03989757, -0.00987033,  0.43144194, -0.11382142],
       [-0.02241018,  0.38008636,  0.13461589, -0.09305646, -0.14219587]]), array([[-0.1890034 ,  0.600126  , -0.7426156 ,  0.21761631,  0.55340511],
       [-0.151529  ,  0.11582031, -0.70261815, -0.32857792,  0.76302686],
       [ 0.77460575,  0.38477601, -0.06873564,  0.4821287 , -0.292996  ],
       [-0.15699172,  0.5356573 ,  0.61318702, -0.1131335 , -0.19171134]]))

dx= [[-0.15771752  0.03091387  0.0333189  -0.06280273 -0.01098687 -0.02298026
   0.1504126  -0.11830679 -0.00556428 -0.03061085]
 [-0.14379227 -0.07935175  0.03498913 -0.17324262  0.00565815  0.02964852
   0.08539494 -0.08853837 -0.03999127 -0.0479425 ]
 [ 0.34977695  0.53057329  0.37713214  0.18460172  0.39459071 -0.03732535
  -0.06595563  0.19999486  0.21331299 -0.08634529]
 [-0.60731354 -